<a href="https://colab.research.google.com/github/JacopoMangiavacchi/NER_Training/blob/main/WNUT17/train_ner_with_explicit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Transformers for Colab

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 2.7MB/s 
     |████████████████████████████████| 890kB 17.2MB/s 
     |████████████████████████████████| 1.1MB 20.1MB/s 
     |████████████████████████████████| 2.9MB 12.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5e4af7ad78b96ddcacb76751b84285898e5d33c486dcb2b10d19a3c34b70a545
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Imports

In [2]:
PRE_TRAINED_BERT_MODEL = 'distilbert-base-cased'

In [3]:
import copy
import math
import warnings

import numpy as np
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

from transformers.activations import gelu
from transformers import DistilBertConfig

from transformers.modeling_outputs import (
    BaseModelOutput,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)

from transformers.modeling_utils import (
    PreTrainedModel,
    apply_chunking_to_forward,
    find_pruneable_heads_and_indices,
    prune_linear_layer,
)

from transformers.utils import logging

from transformers import PreTrainedModel

from transformers import DistilBertPreTrainedModel


# Distilbert For Token Classification

In [4]:
class DistilBertForTokenClassification(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.distilbert(
            input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )



# Get Distilbert for Token Classification model weights

In [5]:
import torch
from transformers import DistilBertForTokenClassification

unique_tags = {
 'B-group',
 'B-product',
 'I-group',
 'B-person',
 'I-product',
 'B-corporation',
 'I-corporation',
 'I-creative-work',
 'B-creative-work',
 'I-location',
 'O',
 'B-location',
 'I-person'
}    

model = DistilBertForTokenClassification.from_pretrained(PRE_TRAINED_BERT_MODEL, num_labels=len(unique_tags))

torch.save(model.state_dict(), PRE_TRAINED_BERT_MODEL + '_weight.pth')

del model

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

# Get data

In [7]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll

--2020-11-06 11:43:09--  http://noisy-text.github.io/2017/files/wnut17train.conll
Resolving noisy-text.github.io (noisy-text.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to noisy-text.github.io (noisy-text.github.io)|185.199.109.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493781 (482K) [application/octet-stream]
Saving to: ‘wnut17train.conll’

wnut17train.conll   100%[===================>] 482.21K  --.-KB/s    in 0.06s   

2020-11-06 11:43:11 (7.29 MB/s) - ‘wnut17train.conll’ saved [493781/493781]



In [8]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [9]:
list(set([str(t) for line in tags for t in line]))

['O',
 'I-group',
 'I-corporation',
 'I-location',
 'I-product',
 'B-corporation',
 'B-creative-work',
 'B-product',
 'B-location',
 'I-creative-work',
 'B-group',
 'I-person',
 'B-person']

In [10]:
print(texts[0][10:17], tags[0][10:17], sep='\n')

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']


In [11]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [12]:
len(train_texts), len(val_texts)

(2715, 679)

In [13]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [14]:
id2tag

{0: 'O',
 1: 'I-group',
 2: 'I-corporation',
 3: 'I-location',
 4: 'I-product',
 5: 'B-corporation',
 6: 'B-creative-work',
 7: 'B-product',
 8: 'B-location',
 9: 'I-creative-work',
 10: 'B-group',
 11: 'I-person',
 12: 'B-person'}

# Use pre-trained DistilBert Tokenizer

In [15]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_BERT_MODEL)
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [16]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])

In [17]:
len(train_encodings['input_ids']), len(val_encodings['input_ids'])

(2715, 679)

In [18]:
len(train_encodings['input_ids'][0])

90

# Format labels for sub token splitting

In [19]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [20]:
len(train_labels), len(val_labels)

(2715, 679)

In [21]:
len(train_labels[0])

90

# Create Dataset loader

In [22]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

# Instantiate Model from PyTorch class module

In [23]:
from transformers import BertConfig
config = BertConfig.from_pretrained(PRE_TRAINED_BERT_MODEL, num_labels=len(unique_tags)) 

In [24]:
model = DistilBertForTokenClassification(config)

# Load Model Weights

In [25]:
weight = torch.load(PRE_TRAINED_BERT_MODEL + '_weight.pth', map_location='cpu')

In [26]:
model.load_state_dict(weight)

<All keys matched successfully>

# Fine Tune DistilBert using PyTorch

In [27]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
    epoch_loss = 0.0
    num_batch = 0
    for i, batch in enumerate(train_loader, 0):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        #print(f"Epoch {epoch} Batch {i} : Loss {loss.data}")
        epoch_loss += loss.item()
        num_batch += 1
    print(f"Epoch {epoch} : Loss {epoch_loss / num_batch}")



Epoch 0 : Loss 0.2988839918857112
Epoch 1 : Loss 0.16127069627537446
Epoch 2 : Loss 0.08903691658257123
Epoch 3 : Loss 0.04722286785904811
Epoch 4 : Loss 0.025664167479365407
Epoch 5 : Loss 0.015710027287166348
Epoch 6 : Loss 0.008975560974367374
Epoch 7 : Loss 0.006832962762393222
Epoch 8 : Loss 0.006154581366076052
Epoch 9 : Loss 0.008136807577530443


# Validate Model

In [28]:
testloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [29]:
predictions = []
actual = []
with torch.no_grad():
   for i, batch in enumerate(testloader, 0):
     input_ids = batch['input_ids'].to(device)
     attention_mask = batch['attention_mask'].to(device)
     outputs = model(input_ids, attention_mask)
     predictions.append(outputs[0].cpu())
     actual.append(batch['labels'])
predictions = np.concatenate(predictions)
actual = np.concatenate(actual)

In [30]:
predicted_tokens = np.argmax(predictions, axis=2)

In [31]:
predicted_tags = []

for p_tokens, a_tokens in zip(predicted_tokens, actual):
  a_mask = a_tokens!=-100
  word_tokens = p_tokens[a_mask]
  iob_tags = [id2tag[x] for x in word_tokens]
  predicted_tags.append(iob_tags)

In [32]:
!pip install seqeval

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=1f832ff19bfbcadf066a76e36e42374bb82fce4c8a5825173822a492a6116fa9
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [33]:
from seqeval.metrics import f1_score

In [34]:
f1_score(val_tags, predicted_tags)

0.5358490566037737